# TRACE 

This is a KG-enhanced reader model that employs Knowledge Graphs (KGs) to imporve generation performance: https://arxiv.org/abs/2406.11460.  

In [1]:
%pip install -q python-terrier accelerate pyterrier_t5
%pip install -q pyterrier_dr 
%pip install -q pyterrier_caching 
# %pip install -q pyterrier-rag

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyterrier as pt
import pyterrier_rag as ptr 

/root/anaconda3/envs/test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Retrieval Setup

Lets get a BM25 retriever from PyTerrier. We could also have used a fast Pisa retriever, via [PyTerrier_Pisa](https://github.com/terrierteam/pyterrier_pisa).

In [ ]:
sparse_index = pt.Artifact.from_hf('pyterrier/raghpq-terrier')
bm25 = sparse_index.bm25(include_fields=['docno', 'text', 'title'])


15:48:57.901 [main] WARN org.terrier.structures.FSADocumentIndex -- This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored


Java started (triggered by TerrierIndex.index_ref) and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


15:48:58.386 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.1 GiB of memory would be required.


# Reader Setup

We're going to use a Llama3-8B-Instruct model through Launcher API as the reader model for generating the final answer.

In [ ]:
from fastchat.model import get_conversation_template
from pyterrier_rag.backend import OpenAIBackend 
from pyterrier_rag.prompt import PromptTransformer
from pyterrier_rag.readers import Reader

LAUNCHER_API_KEY = "put_your_key_here"  # replace with your key 

# let us define the prompt 
system_message = r"""
You are a question answering system that answers based strictly on the provided structured knowledge triples, without using any prior knowledge.

You will be given a list of reasoning paths (each is a sequence of knowledge triples), which together support answering the question.

Instructions:
1. Combine relevant facts across the triples logically.
2. If multiple paths are available, prioritize the top-3 scoring ones.
3. Provide a concise, factual answer. Do NOT explain your reasoning.
4. Do NOT say things like "Based on the context..." or "The chain indicates..."
5. Never hallucinate - only answer what is directly entailed by the reasoning paths.
"""
prompt_text = """
Question: {{ query }}

Reasoning Paths:
{{ qcontext }}

Answer:"""

template = get_conversation_template("meta-llama-3.1-sp")
prompt = PromptTransformer(
    conversation_template=template,
    system_message=system_message,
    instruction=prompt_text,
    api_type="openai"
)

# Now we define the LLM backend used in the reader model 
llm_backend = OpenAIBackend(
    model_id="llama-3-8b-instruct", 
    api_key=LAUNCHER_API_KEY,
    generation_args={"temperature":0.0, "max_tokens":256},
    base_url="http://api.terrier.org/v1"
)

# finally we can obtain a reader 
reader = Reader(llm_backend, prompt)

## TRACE model

TRACE model requires converting each retrieved document into a set of knowledge triples and then constructs KG-based reasoning chains from these triples to identify useful information, we will use the `KnowledgeGraphExtractor` and `ReasoningChainGenerator` to achieve this. 

In [5]:
from pyterrier_rag import KnowledgeGraphExtractor, ReasoningChainGenerator 
from pyterrier_dr import E5 
from pyterrier_caching import ScorerCache # optional 

kg_extractor = KnowledgeGraphExtractor(llm_backend) 
reasoning_chain_generator = ReasoningChainGenerator(
    llm_backend, 
    E5(), 
    dataset="hotpotqa",
    verbose=True
)

cache_path="/nfs/pyterrier/cache" 
kg_cache = ScorerCache(cache_path, kg_extractor, group=None, key="docno", value="knowledge_graph", pickle=True) # Cache the extraced KG triples for later use 
trace = kg_cache >> reasoning_chain_generator >> reader  

bm25_trace = (bm25%10) >> trace # TRACE pipeline with BM25 retrieval 

Initialized ReasoningChainGenerator with E5.base() ranking model


In [6]:
trace 

(Sqlite3ScorerCache('/nfs/pyterrier/cache', <pyterrier_rag.frameworks.kg_rag.kg_extractor.KnowledgeGraphExtractor object at 0x7f7529b1da60>, group='query', key='docno') >> <pyterrier_rag.frameworks.kg_rag.kg_reasoning.ReasoningChainGenerator object at 0x7f7527878440> >> <pyterrier_rag.readers._base.Reader object at 0x7f7698090200>)

# Evaluation 

Lets make a simple RAG pipeline for comparison. 

In [7]:
from pyterrier_rag.prompt import Concatenator

system_message2 = r"""You are an expert Q&A system that is trusted around the world.
        Always answer the query using the provided context information,
        and not prior knowledge.
        rules to follow:
        1. Not directly reference the given context in your answer
        2. Avoid statements like 'Based on the context, ...' or
        'The context information ...' or anything along those lines."""

prompt_text2 = """
Question: {{ query }}

Context information is:{{ qcontext }}

Answer:"""

template = get_conversation_template("meta-llama-3.1-sp")
prompt = PromptTransformer(
    conversation_template=template,
    system_message=system_message2,
    instruction=prompt_text2,
    api_type="openai"
)

# >>Concatenator()
bm25_reader = Reader(llm_backend, prompt=prompt)
bm25 = (bm25%10)>>Concatenator()>>bm25_reader

Now lets run a quick experiment using HotPotQA, comparing vanilla RAG model and TRACE. 

In [9]:
dataset = pt.get_dataset('rag:hotpotqa')
pt.Experiment(
    [bm25, bm25_trace],
    dataset.get_topics('dev').head(50),
    dataset.get_answers('dev'),
    [ptr.measures.F1, ptr.measures.EM],
    batch_size=25,
    verbose=True,
    precompute_prefix=True,
    names=['RAG', 'TRACE'],
    baseline=0
)

Precomputing results of 50 topics on shared pipeline component TerrierRetr(BM25)
/root/anaconda3/envs/test/lib/python3.12/site-packages/pyterrier/_evaluation/_execution.py:311: UserWarning: precompute_prefix with batch_size is very experimental. Please report any problems
  warn("precompute_prefix with batch_size is very experimental. Please report any problems")
pt.Experiment: 100%|██████████| 4/4 [04:24<00:00, 66.08s/batches]


,name,F1,EM,F1 +,F1 -,F1 p-value,EM +,EM -,EM p-value
0,RAG,0.164343,0.02,NaN,NaN,NaN,NaN,NaN,NaN
1,TRACE,0.281303,0.18,13.0,11.0,0.055328,8.0,0.0,0.003635
